# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-07 08:41:05] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=37370, nccl_port=None, mem_fraction_static=0.849, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1029250374, constrained_json_

[2025-07-07 08:41:06] Inferred chat template from model path: llama-2


[2025-07-07 08:41:17] Attention backend not set. Use flashinfer backend by default.
[2025-07-07 08:41:17] Init torch distributed begin.


[2025-07-07 08:41:17] Init torch distributed ends. mem usage=0.00 GB


[2025-07-07 08:41:19] Load weight begin. avail mem=61.98 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.32s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.32s/it]

[2025-07-07 08:41:24] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=37.72 GB, mem usage=0.93 GB.
[2025-07-07 08:41:24] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-07-07 08:41:24] Memory pool end. avail mem=37.40 GB


[2025-07-07 08:41:25] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=37.63 GB


[2025-07-07 08:41:25] INFO:     Started server process [552226]
[2025-07-07 08:41:25] INFO:     Waiting for application startup.
[2025-07-07 08:41:25] INFO:     Application startup complete.
[2025-07-07 08:41:25] INFO:     Uvicorn running on http://127.0.0.1:37370 (Press CTRL+C to quit)


[2025-07-07 08:41:26] INFO:     127.0.0.1:46364 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-07 08:41:26] INFO:     127.0.0.1:46370 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-07 08:41:26] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T08:41:26.626139


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-07 08:42:31] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, #token: 7, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-07T08:42:31.920856


[2025-07-07 08:43:01] INFO:     127.0.0.1:46378 - "POST /generate HTTP/1.1" 200 OK
[2025-07-07 08:43:01] The server is fired up and ready to roll!


[2025-07-07 08:43:04] INFO:     127.0.0.1:46392 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-07-07 08:43:04] Child process unexpectedly failed with exitcode=9. pid=553282


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-07 08:43:19] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=39896, nccl_port=None, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=191631288, constrained_json_whi

[2025-07-07 08:43:19] Inferred chat template from model path: llama-2


[2025-07-07 08:43:30] Attention backend not set. Use flashinfer backend by default.
[2025-07-07 08:43:30] Init torch distributed begin.


[2025-07-07 08:43:30] Init torch distributed ends. mem usage=0.00 GB


[2025-07-07 08:43:32] Load weight begin. avail mem=41.65 GB


...

[2025-07-07 08:43:38] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=54.10 GB


[2025-07-07 08:43:38] INFO:     Started server process [559782]
[2025-07-07 08:43:38] INFO:     Waiting for application startup.
[2025-07-07 08:43:38] INFO:     Application startup complete.
[2025-07-07 08:43:38] INFO:     Uvicorn running on http://127.0.0.1:39896 (Press CTRL+C to quit)


[2025-07-07 08:43:39] INFO:     127.0.0.1:41828 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-07 08:43:39] INFO:     127.0.0.1:41838 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-07 08:43:39] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T08:43:39.738434


[2025-07-07 08:43:42] INFO:     127.0.0.1:41842 - "POST /generate HTTP/1.1" 200 OK
[2025-07-07 08:43:42] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-07 08:43:44] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T08:43:44.531003


[2025-07-07 08:43:44] INFO:     127.0.0.1:58584 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-07-07 08:43:44] Child process unexpectedly failed with exitcode=9. pid=560078
[2025-07-07 08:43:44] Child process unexpectedly failed with exitcode=9. pid=559944


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-07 08:43:56] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=30791, nccl_port=None, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=728760202,

[2025-07-07 08:44:14] Casting torch.bfloat16 to torch.float16.


[2025-07-07 08:44:15] Casting torch.bfloat16 to torch.float16.
[2025-07-07 08:44:15] Attention backend not set. Use flashinfer backend by default.
[2025-07-07 08:44:15] Init torch distributed begin.


[2025-07-07 08:44:17] Init torch distributed ends. mem usage=0.01 GB


[2025-07-07 08:44:18] Load weight begin. avail mem=77.99 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.17s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.17s/it]

[2025-07-07 08:44:35] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=42.00 GB, mem usage=1.80 GB.
[2025-07-07 08:44:35] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-07-07 08:44:35] Memory pool end. avail mem=41.92 GB


[2025-07-07 08:44:36] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=42.10 GB


[2025-07-07 08:44:36] INFO:     Started server process [561923]
[2025-07-07 08:44:36] INFO:     Waiting for application startup.
[2025-07-07 08:44:36] INFO:     Application startup complete.
[2025-07-07 08:44:36] INFO:     Uvicorn running on http://127.0.0.1:30791 (Press CTRL+C to quit)


[2025-07-07 08:44:37] INFO:     127.0.0.1:48594 - "GET /v1/models HTTP/1.1" 200 OK
[2025-07-07 08:44:37] INFO:     127.0.0.1:48600 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-07 08:44:37] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T08:44:37.983655


[2025-07-07 08:44:40] INFO:     127.0.0.1:48602 - "POST /generate HTTP/1.1" 200 OK
[2025-07-07 08:44:40] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-07 08:44:43] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T08:44:43.025893


[2025-07-07 08:44:43] INFO:     127.0.0.1:48608 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-07-07 08:44:43] Child process unexpectedly failed with exitcode=9. pid=562456


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-07 08:44:53] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=31980, nccl_port=None, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=57373421, constrain

[2025-07-07 08:45:05] Casting torch.bfloat16 to torch.float16.


[2025-07-07 08:45:06] Casting torch.bfloat16 to torch.float16.
[2025-07-07 08:45:06] Attention backend not set. Use flashinfer backend by default.
[2025-07-07 08:45:06] Init torch distributed begin.


[2025-07-07 08:45:06] Init torch distributed ends. mem usage=0.04 GB


[2025-07-07 08:45:08] Load weight begin. avail mem=60.25 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.78it/s]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.78it/s]

[2025-07-07 08:45:26] Load weight end. type=LlamaForCausalLMEagle3, dtype=torch.float16, avail mem=57.97 GB, mem usage=2.15 GB.
[2025-07-07 08:45:26] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-07-07 08:45:26] Memory pool end. avail mem=57.88 GB


[2025-07-07 08:45:26] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=58.56 GB


[2025-07-07 08:45:27] INFO:     Started server process [564170]
[2025-07-07 08:45:27] INFO:     Waiting for application startup.
[2025-07-07 08:45:27] INFO:     Application startup complete.
[2025-07-07 08:45:27] INFO:     Uvicorn running on http://127.0.0.1:31980 (Press CTRL+C to quit)


[2025-07-07 08:45:28] INFO:     127.0.0.1:34280 - "GET /v1/models HTTP/1.1" 200 OK
[2025-07-07 08:45:28] INFO:     127.0.0.1:34292 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-07 08:45:28] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T08:45:28.472035


[2025-07-07 08:45:31] INFO:     127.0.0.1:34308 - "POST /generate HTTP/1.1" 200 OK
[2025-07-07 08:45:31] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-07 08:45:33] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T08:45:33.527398


[2025-07-07 08:45:33] INFO:     127.0.0.1:34318 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-07-07 08:45:33] Child process unexpectedly failed with exitcode=9. pid=564856


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-07 08:45:45] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='XiaomiMiMo/MiMo-7B-RL', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=34738, nccl_port=None, mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1057026484, constrained_json_whitespace_pattern=None, watc

[2025-07-07 08:45:56] Attention backend not set. Use flashinfer backend by default.
[2025-07-07 08:45:56] Init torch distributed begin.


[2025-07-07 08:45:56] Init torch distributed ends. mem usage=0.00 GB


[2025-07-07 08:45:57] Load weight begin. avail mem=78.50 GB


[2025-07-07 08:45:57] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-07-07 08:46:03] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-07-07 08:46:04] INFO:     Started server process [565964]
[2025-07-07 08:46:04] INFO:     Waiting for application startup.
[2025-07-07 08:46:04] INFO:     Application startup complete.
[2025-07-07 08:46:04] INFO:     Uvicorn running on http://0.0.0.0:34738 (Press CTRL+C to quit)


[2025-07-07 08:46:04] INFO:     127.0.0.1:53202 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-07 08:46:05] INFO:     127.0.0.1:53206 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-07 08:46:05] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T08:46:05.490788


[2025-07-07 08:46:06] INFO:     127.0.0.1:53214 - "POST /generate HTTP/1.1" 200 OK
[2025-07-07 08:46:06] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-07-07 08:46:09] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-07T08:46:09.921655


[2025-07-07 08:46:10] Decode batch. #running-req: 1, #token: 93, token usage: 0.00, accept len: 1.81, cuda graph: False, gen throughput (token/s): 10.77, #queue-req: 0, timestamp: 2025-07-07T08:46:10.512998


[2025-07-07 08:46:11] Decode batch. #running-req: 1, #token: 162, token usage: 0.01, accept len: 1.73, cuda graph: False, gen throughput (token/s): 109.36, #queue-req: 0, timestamp: 2025-07-07T08:46:11.143935


[2025-07-07 08:46:11] Decode batch. #running-req: 1, #token: 232, token usage: 0.01, accept len: 1.75, cuda graph: False, gen throughput (token/s): 109.31, #queue-req: 0, timestamp: 2025-07-07T08:46:11.784327
[2025-07-07 08:46:11] INFO:     127.0.0.1:53218 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-07-07 08:46:11] Child process unexpectedly failed with exitcode=9. pid=566247


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).